In [1]:
from google.colab import drive
drive.mount('/gdrive')
!cp -R /content/../gdrive/My\ Drive/MHC/Project/Data/ -d ./

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [1]:
from __future__ import print_function

%matplotlib inline
import pandas as pd
import numpy as np
from math import log
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from scipy.sparse import * 
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer, TfidfTransformer
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
import math

from hyperopt import hp, tpe
from hyperopt.fmin import fmin

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
import lightgbm as lgb
import gc
from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time
import json

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
def load_data():
  df1 = pd.read_csv('./Data/train1.csv', header = None)
  df2 = pd.read_csv('./Data/test1.csv', header = None)
  df = df1.append(df2)
  df.drop_duplicates(inplace = True)
  df.reset_index(inplace = True, drop = True)
  df.columns = ['peptide', 'aff', 'hla']
  df = df.groupby(['peptide','hla']).mean().reset_index()
  df = df.groupby('hla').filter(lambda x : len(x)>=30).reset_index(drop = True)
  
  return df
df = load_data()

In [3]:
def labelenc(df, enc):
    enc = LabelEncoder().fit(df.iloc[:,0])
    for c in df.columns:
        df[c] = enc.transform(df[c])
    
    return df

def tear(serie, n, col):
    clus_df = pd.DataFrame(serie.copy())
    for i in range(1,n+1):
        col_name = col + '_P' + str(i)
        clus_df[col_name] = clus_df[col].apply(lambda x : x[i-1])

    cols = [c for c in clus_df.columns if c not in [col]]

    return clus_df[cols]

def hla_preprocess(df):
    df['allele_type'] = df['hla'].apply(
    lambda x: (x.startswith('DRB') and x[0:3]) or (x.startswith('HLA-DQ') and x[0:6]) or (x.startswith('HLA-DP') and x[0:6]) or x[0:3])

    hla_encoder = LabelEncoder()
    #hla_encoder.fit(df['hla'])
    #np.save('Data/hla_encoder_classes.npy', hla_encoder.classes_)
    hla_encoder.classes_ = np.load('./Data/hla_encoder_classes.npy')
    df['hla'] = hla_encoder.transform(df['hla'])

    allele_type_encoder = LabelEncoder()
    #allele_type_encoder.fit(df['allele_type'])
    #np.save('Data/allele_type_encoder_classes.npy', allele_type_encoder.classes_)
    allele_type_encoder.classes_ = np.load('./Data/allele_type_encoder_classes.npy')
    df['allele_type'] = allele_type_encoder.transform(df['allele_type'])

    return df
df = hla_preprocess(df)
df['length'] = df['peptide'].apply(lambda x: len(x))

In [4]:
def multiply(df):
    n_iteration = np.int32(df['length'] - 9 +1)
    df_result = df.loc[np.repeat(df.index.values,n_iteration)]
    df_result.reset_index(inplace = True)
    df_result.rename(columns = {'index':'true_index'}, inplace = True)

    return df_result
df = multiply(df)

In [5]:
def L_impute_space(s):
    k = 3 - len(s)
    return ' '* k + s

def R_impute_space(s):
    k = 3 - len(s)
    return s + ' '* k
    
def window_select(k):
    #tqdm.pandas()
    k['true_index'] += 1
    k['n_label'] = k[['true_index', 'peptide','hla']].groupby(['peptide','hla']).agg('cumsum')
    k['n_label'] /= k['true_index']
    k['n_label'] = k['n_label'].map(int) - 1
    
    print('String Distinguishing...')
    k['core'] = k[['peptide', 'n_label']].apply(lambda x : x['peptide'][x['n_label']:x['n_label'] + 9], axis=1)
    k['LPFR'] = k[['peptide', 'n_label']].apply(lambda x : x['peptide'][max(x['n_label'] - 3, 0): x['n_label']], axis=1)
    k['RPFR'] = k[['peptide', 'n_label','length']].apply(lambda x : x['peptide'][x['n_label'] + 9 : min(x['n_label'] + 9 + 3, x['length'])], axis=1)
    print('Done String Distinguishing...')
    
    print('String Transforming...')
    k['len_LPFR'] = k['LPFR'].apply(lambda x: len(x)/3)
    k['len_RPFR'] = k['RPFR'].apply(lambda x: len(x)/3)
    k['len_peptide_transform'] = k['length'].apply(lambda x : 1/(1+np.exp((x - 15)/2)))
    
    k['LPFR'] = k['LPFR'].apply(lambda x:L_impute_space(x))
    k['RPFR'] = k['RPFR'].apply(lambda x:L_impute_space(x))
    return k

df = window_select(df)

String Distinguishing...


KeyboardInterrupt: 

In [ ]:
def tfidf(df, col):
    vectorizer = TfidfVectorizer(analyzer = 'char')
    X = np.array(vectorizer.fit_transform(df[col]).toarray(), dtype=np.float16)
    #pickle.dump(vectorizer.fit_transform(df[col]), open("tfidf.pickle", "wb"))
    for i in range(len(vectorizer.get_feature_names())):
        df[col + '_' + vectorizer.get_feature_names()[i] + '_tfidf'] = X[:, i]
    
    return df

for col in tqdm(['core','LPFR', 'RPFR']):
    df = pd.concat([df, labelenc(tear(df[col], df[col].map(len).max(), col), LabelEncoder().fit(tear(df[col], df[col].map(len).max(), col).values.flatten()))], axis=1)
df = tfidf(df, 'core')

In [ ]:
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=10000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()

def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')
  
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)
    
    
k = df['core'].apply(lambda x: " ".join(x))
dataset = k


#labels = dataset.target
true_k = int(math.sqrt(k.nunique()))

print("Extracting features from the training dataset "
      "using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features,analyzer = 'char',
                                   stop_words='english', alternate_sign=False,
                                   norm=None, binary=False)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(n_features=opts.n_features,
                                       stop_words='english',analyzer = 'char',
                                       alternate_sign=False, norm='l2',
                                       binary=False)
else:
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                 min_df=2, stop_words='english',analyzer = 'char',
                                 use_idf=opts.use_idf)
X = vectorizer.fit_transform(dataset)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()


# #############################################################################
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000, verbose=opts.verbose,
                        random_state = 42)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=opts.verbose, random_state = 42)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))


'''if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()'''
        
df['cluster'] = km.labels_

In [ ]:
def mean_encoding(df):
    groupby_list = ['hla', 'allele_type', 'core_P1', 'core_P2', 'core_P3', 'core_P4',
                    'core_P5', 'core_P6', 'core_P7', 'core_P8', 'core_P9', 'LPFR_P1',
                    'LPFR_P2', 'LPFR_P3', 'RPFR_P1', 'RPFR_P2', 'RPFR_P3', 'cluster',]

    for cat in tqdm(groupby_list):
        tmp = df.groupby(cat).agg({'aff':['mean', 'std']})

        tmp.columns = [cat + '_' + s for s in ['mean', 'std']]
        tmp[cat] = tmp.index
        df = pd.merge(df, tmp, on = cat, how = 'left')
        
    return df
  
df = mean_encoding(df)
df['peptide_nunique'] = df['peptide'].apply(lambda x: len(''.join(set(x))))
df['unique_rate'] = df['peptide_nunique']/ df['length']

In [10]:
with open('Data/blosum62.json') as json_data:
    d = json.load(json_data)
output = {
    'p1':[],
    'p2':[],
    'p3':[],
    'p4':[],
    'p5':[],
    'p6':[],
    'p7':[],
    'p8':[],
    'p9':[],
}

for string in tqdm(df['core']):
    cnt = 1
    for s in string:
        tmp = 0
        for k in string:
            tmp += d[s][k]
        loc = 'p'+str(cnt)
        cnt += 1
        output[loc].append(tmp)

100%|██████████| 975202/975202 [00:20<00:00, 47434.95it/s]


In [ ]:
print(df.shape)
df = pd.concat([df,pd.DataFrame(output)], axis = 1)
df['blosum_sum'] = df['p1'] + df['p2'] + df['p3'] + df['p4'] + df['p5'] + df['p6'] + df['p7'] + df['p8'] + df['p9']
print(df.shape)

In [8]:
import pickle
feature_engineered = {
  'hla':['hla', 'allele_type'],
  'length':['length'],
  'len_PFR':['len_LPFR', 'len_RPFR', 'len_peptide_transform'],
  'core':['core_P1', 'core_P2', 'core_P3', 'core_P4', 'core_P5', 'core_P6', 'core_P7', 'core_P8', 'core_P9'],
  'PFR':['LPFR_P1', 'LPFR_P2', 'LPFR_P3', 'RPFR_P1', 'RPFR_P2', 'RPFR_P3'],
  'core_tfidf':[],
  'core_count':[],
  'cluster':['cluster'],
  'unique':['peptide_nunique','unique_rate'],
  'blosum_sum':['blosum_sum']

}
def countvec(df, col, json):
  with open("./Data/count_vectorizer.pickle", "rb") as input_file:
    vectorizer = pickle.load(input_file)
  X = np.array(vectorizer.fit_transform(list(df['core'])).toarray(), dtype=np.float16)

  for i in range(len(vectorizer.get_feature_names())):
    name = str(col) + '_' + str(vectorizer.get_feature_names()[i]) + '_count'
    df[name] = X[:, i]
    json['core_count'].append(name)
  print('countvec...done')
  return df, json

df, ace = countvec(df, 'core', feature_engineered)

countvec...done


In [9]:
feature_engineered

{'hla': ['hla', 'allele_type'],
 'length': ['length'],
 'len_PFR': ['len_LPFR', 'len_RPFR', 'len_peptide_transform'],
 'core': ['core_P1',
  'core_P2',
  'core_P3',
  'core_P4',
  'core_P5',
  'core_P6',
  'core_P7',
  'core_P8',
  'core_P9'],
 'PFR': ['LPFR_P1', 'LPFR_P2', 'LPFR_P3', 'RPFR_P1', 'RPFR_P2', 'RPFR_P3'],
 'core_tfidf': [],
 'core_count': ['core_a_count',
  'core_c_count',
  'core_d_count',
  'core_e_count',
  'core_f_count',
  'core_g_count',
  'core_h_count',
  'core_i_count',
  'core_k_count',
  'core_l_count',
  'core_m_count',
  'core_n_count',
  'core_p_count',
  'core_q_count',
  'core_r_count',
  'core_s_count',
  'core_t_count',
  'core_v_count',
  'core_w_count',
  'core_y_count'],
 'cluster': ['cluster'],
 'unique': ['peptide_nunique', 'unique_rate'],
 'blosum_sum': ['blosum_sum']}

In [39]:
from sklearn.feature_selection import RFECV

cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label']

X = df.drop(columns= cols_to_drop, axis = 1)
y = df['aff'].apply(lambda x: 1 if x >= 0.5 else 0)

In [44]:
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier()
selector = RFECV(estimator, step=1, cv=5, verbose = 1, n_jobs = -1)
selector = selector.fit(X, y)
selector.support_

Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estimator with 86 features.
Fitting estimator with 85 features.
Fitting estimator with 84 features.
Fitting estimator with 83 features.
Fitting estimator with 82 features.
Fitting estimator with 81 features.
Fitting estimator with 80 features.
Fitting estimator with 79 features.
Fitting estimator with 78 features.
Fitting estimator with 77 features.
Fitting estimator with 76 features.
Fitting estimator with 75 features.
Fitting estimator with 74 features.
Fitting estimator with 73 features.
Fitting estimator with 72 features.
Fitting estimator with 71 features.
Fitting estimator with 70 features.
Fitting estimator with 69 features.
Fitting estimator with 68 features.
Fitting estimator with 67 features.
Fitting estimator with 66 features.
Fitting estimator with 65 features.
Fitting estimator with 64 features.
Fitting estimator with 63 fe

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False,  True, False, False, False,
        True, False,  True, False,  True,  True, False,  True, False,
       False,  True,  True,  True, False, False, False, False,  True])

In [47]:
X.columns[selector.support_]

Index(['hla', 'core_a_tfidf', 'core_l_tfidf', 'cluster', 'hla_mean', 'hla_std',
       'LPFR_P2_mean', 'LPFR_P3_mean', 'RPFR_P2_mean', 'RPFR_P3_mean',
       'cluster_mean', 'cluster_std', 'unique_rate', 'p3', 'p4', 'p5',
       'blosum_sum'],
      dtype='object')

In [ ]:
from sklearn.svm import SVC
estimator = SVC()
svc_selector = RFECV(estimator, step=1, cv=5, verbose = 1, n_jobs = -1).fit(X, y)
X.columns[svc_selector.support_]

In [20]:
X.columns[svc_selector.support_]

NameError: name 'svc_selector' is not defined

In [18]:
set(X.columns) - set(['hla', 'allele_type', 'length', 'len_LPFR', 'len_RPFR',
       'len_peptide_transform', 'core_P1', 'core_P2', 'core_P3', 'core_P4',
       'core_P5', 'core_P6', 'core_P7', 'core_P8', 'core_P9', 'LPFR_P1',
       'LPFR_P2', 'RPFR_P1', 'RPFR_P2', 'RPFR_P3', 'core_a_tfidf',
       'core_c_tfidf', 'core_d_tfidf', 'core_e_tfidf', 'core_f_tfidf',
       'core_g_tfidf', 'core_h_tfidf', 'core_i_tfidf', 'core_k_tfidf',
       'core_l_tfidf', 'core_m_tfidf', 'core_n_tfidf', 'core_p_tfidf',
       'core_q_tfidf', 'core_r_tfidf', 'core_s_tfidf', 'core_t_tfidf',
       'core_v_tfidf', 'core_w_tfidf', 'core_y_tfidf', 'cluster', 'hla_mean',
       'hla_std', 'allele_type_mean', 'allele_type_std', 'core_P1_mean',
       'core_P1_std', 'core_P2_mean', 'core_P2_std', 'core_P3_mean',
       'core_P3_std', 'core_P4_mean', 'core_P4_std', 'core_P5_mean',
       'core_P5_std', 'core_P6_mean', 'core_P6_std', 'core_P7_mean',
       'core_P7_std', 'core_P8_mean', 'core_P8_std', 'core_P9_mean',
       'core_P9_std', 'LPFR_P1_mean', 'LPFR_P1_std', 'LPFR_P2_mean',
       'LPFR_P2_std', 'LPFR_P3_mean', 'LPFR_P3_std', 'RPFR_P1_mean',
       'RPFR_P1_std', 'RPFR_P2_mean', 'RPFR_P2_std', 'RPFR_P3_mean',
       'RPFR_P3_std', 'cluster_mean', 'cluster_std', 'peptide_nunique',
       'unique_rate', 'p1', 'p2', 'p5', 'p6', 'p7', 'p8', 'p9', 'blosum_sum'])

{'LPFR_P3', 'p3', 'p4'}

In [49]:
from sklearn.linear_model import RidgeClassifier
estimator = RidgeClassifier()
RidgeClassifier_selector = RFECV(estimator, step=1, cv=5, verbose = 1, n_jobs = -1).fit(X, y)
X.columns[RidgeClassifier_selector.support_]

Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.


Index(['hla', 'allele_type', 'length', 'len_LPFR', 'len_RPFR',
       'len_peptide_transform', 'core_P1', 'core_P2', 'core_P3', 'core_P4',
       'core_P5', 'core_P6', 'core_P7', 'core_P8', 'core_P9', 'LPFR_P1',
       'LPFR_P2', 'RPFR_P1', 'RPFR_P2', 'RPFR_P3', 'core_a_tfidf',
       'core_c_tfidf', 'core_d_tfidf', 'core_e_tfidf', 'core_f_tfidf',
       'core_g_tfidf', 'core_h_tfidf', 'core_i_tfidf', 'core_k_tfidf',
       'core_l_tfidf', 'core_m_tfidf', 'core_n_tfidf', 'core_p_tfidf',
       'core_q_tfidf', 'core_r_tfidf', 'core_s_tfidf', 'core_t_tfidf',
       'core_v_tfidf', 'core_w_tfidf', 'core_y_tfidf', 'cluster', 'hla_mean',
       'hla_std', 'allele_type_mean', 'allele_type_std', 'core_P1_mean',
       'core_P1_std', 'core_P2_mean', 'core_P2_std', 'core_P3_mean',
       'core_P3_std', 'core_P4_mean', 'core_P4_std', 'core_P5_mean',
       'core_P5_std', 'core_P6_mean', 'core_P6_std', 'core_P7_mean',
       'core_P7_std', 'core_P8_mean', 'core_P8_std', 'core_P9_mean',
       'co

In [50]:
from sklearn.linear_model import SGDClassifier
estimator = SGDClassifier()
SGDClassifier_selector = RFECV(estimator, step=1, cv=5, verbose = 1, n_jobs = -1).fit(X, y)
X.columns[SGDClassifier_selector.support_]

Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estimator with 86 features.
Fitting estimator with 85 features.
Fitting estimator with 84 features.
Fitting estimator with 83 features.
Fitting estimator with 82 features.
Fitting estimator with 81 features.
Fitting estimator with 80 features.
Fitting estimator with 79 features.
Fitting estimator with 78 features.
Fitting estimator with 77 features.
Fitting estimator with 76 features.
Fitting estimator with 75 features.
Fitting estimator with 74 features.
Fitting estimator with 73 features.
Fitting estimator with 72 features.
Fitting estimator with 71 features.
Fitting estimator with 70 features.
Fitting estimator with 69 features.
Fitting estimator with 68 features.
Fitting estimator with 67 features.
Fitting estimator with 66 features.
Fitting estimator with 65 features.
Fitting estimator with 64 features.
Fitting estimator with 63 fe

Index(['len_peptide_transform', 'core_a_tfidf', 'core_c_tfidf', 'core_e_tfidf',
       'core_i_tfidf', 'core_k_tfidf', 'core_l_tfidf', 'core_n_tfidf',
       'hla_mean', 'hla_std', 'allele_type_mean', 'allele_type_std',
       'core_P1_mean', 'core_P1_std', 'core_P2_std', 'core_P3_mean',
       'core_P4_mean', 'core_P4_std', 'core_P5_mean', 'core_P5_std',
       'core_P6_mean', 'core_P6_std', 'core_P7_mean', 'core_P7_std',
       'core_P8_mean', 'core_P8_std', 'core_P9_mean', 'core_P9_std',
       'LPFR_P1_mean', 'LPFR_P1_std', 'LPFR_P2_mean', 'LPFR_P3_mean',
       'LPFR_P3_std', 'RPFR_P1_mean', 'RPFR_P1_std', 'RPFR_P2_mean',
       'RPFR_P2_std', 'RPFR_P3_mean', 'RPFR_P3_std', 'cluster_mean',
       'unique_rate'],
      dtype='object')

In [51]:
from sklearn.neural_network import MLPClassifier
estimator = MLPClassifier()
MLPClassifier_selector = RFECV(estimator, step=1, cv=5, verbose = 1, n_jobs = -1).fit(X, y)
X.columns[MLPClassifier_selector.support_]

RuntimeError: The classifier does not expose "coef_" or "feature_importances_" attributes

In [143]:
cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label']

X = df.drop(columns= cols_to_drop, axis = 1)
y = df['aff'].apply(lambda x: 1 if x >= 0.5 else 0)

def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'bagging_fraction': params['bagging_fraction'],
        'bagging_freq': int(params['bagging_freq']),
        'feature_fraction': params['feature_fraction'],
        'max_bin': int(params['max_bin']),
        'boosting': str(params['boosting']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'min_sum_hessian_in_leaf': params['min_sum_hessian_in_leaf'],
        'min_gain_to_split': params['min_gain_to_split'],
        'lambda_l2': params['lambda_l2'],
        'lambda_l1': params['lambda_l1'],
        'scale_pos_weight': params['scale_pos_weight'],
        'learning_rate': 0.01,
        'num_iterations':200,
        'seed':0,
        
    }
    
    clf = lgb.LGBMClassifier(
        #num_boost_round = 500,
        #learning_rate = 0.01,
        **params
    )
    
    score = cross_val_score(clf, X, y, scoring='roc_auc', cv=KFold(n_splits = 5)).mean()
    #print("AUC {:.3f} params {}".format(score, params))
    return -score

space = {
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'max_depth': hp.randint('max_depth', 9) + 1,
    'min_data_in_leaf': hp.quniform('min_data_in_leaf',100, 1000, 2),
    'num_leaves': int(hp.quniform('num_leaves', 8, 128, 2)),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.7, 0.95),
    'bagging_freq': hp.randint('bagging_freq', 9) + 1,
    'feature_fraction': hp.uniform('feature_fraction', 0.7, 0.95),
    'max_bin': hp.randint('max_bin', 255) + 100,
    'boosting': hp.choice('boosting', ["dart", "gbdt"]),
    'min_data_in_leaf': hp.randint('min_data_in_leaf', 20) + 10,
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 1e-4, 1e-2),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 5, 50),
    'lambda_l2': hp.uniform('lambda_l2', 0, 1),
    'lambda_l1': hp.uniform('lambda_l1', 0, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.1, 1),
    
}

best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=1000
)

import json
with open('best_param.json', 'w') as outfile:
    json.dump(best_params, outfile)

In [26]:
cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label']

#df.rename(columns={'aff_x':'aff'},  inplace = True)
X = df.drop(columns= cols_to_drop, axis = 1)
#X = df[['hla_mean', 'hla', 'cluster_mean', 'length', 'unique_rate', 'cluster_std']]
y = df['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)

auc_score = []
FI = []
N_FOLDS = 5

oof_preds = np.zeros(X.shape[0])
#sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in X.columns]

for i in range(N_FOLDS):
    max_index = df.true_index.max()
    k = math.floor(max_index/N_FOLDS)
    if i == N_FOLDS-1:
        train_index, test_index = list(set(df.index) - set(df[df['true_index'].between(k*i+1, max_index, inclusive=True)].index)), list(set(df[df['true_index'].between(k*i+1, max_index, inclusive=True)].index))
    else: 
        train_index, test_index = list(set(df.index) - set(df[df['true_index'].between(k*i+1, k*(i+1), inclusive=True)].index)), list(set(df[df['true_index'].between(k*i+1, k*(i+1), inclusive=True)].index))
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    # Parameters
    MAX_BOOST_ROUNDS = 100000
    LEARNING_RATE = 0.01

    #x_train = x_train.values.astype(np.float32, copy=False)
    d_train = lgb.Dataset(X_train, label= y_train)
    d_valid = lgb.Dataset(X_test, label = y_test)
    # Params
    params = {
        'objective':'binary',
        'metric': 'auc',
        "boosting": 'gbdt', 
        'learning_rate': LEARNING_RATE,
        'seed': 0,
        #'is_unbalance': True,
    }
    #Model
    clf = lgb.train(
            params=params,
            train_set=d_train,
            num_boost_round = MAX_BOOST_ROUNDS,
            valid_sets=[d_train, d_valid],
            early_stopping_rounds=200,
            verbose_eval=500
        )
    
    oof_preds[test_index] = clf.predict(X_test)
'''    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = N_FOLDS + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
feature_importance_df.to_csv('feature_importance_df_no_core.csv', index = False)'''
#!cp ./feature_importance_df2.csv -d /content/../gdrive/My\ Drive/MHC/Project/Data/ 

Training until validation scores don't improve for 200 rounds.
[500]	training's auc: 0.816523	valid_1's auc: 0.785778
[1000]	training's auc: 0.842899	valid_1's auc: 0.809729
[1500]	training's auc: 0.857626	valid_1's auc: 0.819999
[2000]	training's auc: 0.868209	valid_1's auc: 0.825767
[2500]	training's auc: 0.877032	valid_1's auc: 0.83013
[3000]	training's auc: 0.883801	valid_1's auc: 0.833065
[3500]	training's auc: 0.889998	valid_1's auc: 0.835088
[4000]	training's auc: 0.895786	valid_1's auc: 0.836218
[4500]	training's auc: 0.901078	valid_1's auc: 0.837176
[5000]	training's auc: 0.905948	valid_1's auc: 0.838086


KeyboardInterrupt: 

In [27]:
cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label']

X = df.drop(columns= cols_to_drop, axis = 1).sample(frac=1).reset_index(drop = True)
#X = df[['hla_mean', 'hla', 'cluster_mean', 'length', 'unique_rate', 'cluster_std']]
y = df['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)

auc_score = []
FI = []
N_FOLDS = 5

oof_preds = np.zeros(X.shape[0])
#sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in X.columns]

for train_index, test_index in KFold(n_splits=N_FOLDS).split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    # Parameters
    MAX_BOOST_ROUNDS = 100000
    LEARNING_RATE = 0.01

    #x_train = x_train.values.astype(np.float32, copy=False)
    d_train = lgb.Dataset(X_train, label= y_train)
    d_valid = lgb.Dataset(X_test, label = y_test)
    # Params
    params = {
        'objective':'binary',
        'metric': 'auc',
        "boosting": 'gbdt', 
        'learning_rate': LEARNING_RATE,
        'seed': 0,
        #'is_unbalance': True,
    }
    #Model
    clf = lgb.train(
            params=params,
            train_set=d_train,
            num_boost_round = MAX_BOOST_ROUNDS,
            valid_sets=[d_train, d_valid],
            early_stopping_rounds=200,
            verbose_eval=500
        )
    
    oof_preds[test_index] = clf.predict(X_test)
'''    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = N_FOLDS + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
feature_importance_df.to_csv('feature_importance_df_no_core.csv', index = False)'''
#!cp ./feature_importance_df2.csv -d /content/../gdrive/My\ Drive/MHC/Project/Data/ 

Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[2]	training's auc: 0.50888	valid_1's auc: 0.501062
Training until validation scores don't improve for 200 rounds.


KeyboardInterrupt: 

In [67]:
clf.save_model('mode.txt')
bst = lgb.Booster(model_file='mode.txt')
bst.predict(X_test)

array([0.2181161 , 0.22773271, 0.24020651, ..., 0.88992833, 0.89454248,
       0.87875922])

In [71]:
roc_auc_score(k['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0), k['oof_preds'])

0.8480222721742423

In [69]:
#gbdt all feature + vector -p_number
df['oof_preds'] = oof_preds
idx = df.groupby(['true_index'])['oof_preds'].transform(max) == df['oof_preds']
k = df[idx]
df.drop(columns = 'oof_preds', inplace = True)
from sklearn.metrics import roc_auc_score
roc_auc_score(k['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0), k['oof_preds'])
k.drop(columns = ['aff', 'true_index','n_label']).to_csv('stage2_input2.csv',index = False)

In [70]:
k.shape

(133348, 98)

In [28]:
#goss
'''cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label', 'core_P1', 'core_P2', 'core_P3', 'core_P4',
                    'core_P5', 'core_P6', 'core_P7', 'core_P8', 'core_P9','allele_type', 'len_peptide_transform',
                'core_P1_mean', 'core_P1_std', 'core_P2_std', 'core_P3_mean',
                 'core_P4_mean', 'core_P4_std', 'core_P5_mean', 'core_P5_std',
                 'core_P6_mean', 'core_P6_std', 'core_P7_mean', 'core_P7_std',
                 'core_P8_mean', 'core_P8_std', 'core_P9_mean', 'core_P9_std',]'''
df['oof_preds'] = oof_preds
idx = df.groupby(['true_index'])['oof_preds'].transform(max) == df['oof_preds']
k = df[idx]
df.drop(columns = 'oof_preds', inplace = True)
from sklearn.metrics import roc_auc_score
roc_auc_score(k['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0), k['oof_preds'])
#df.drop(columns='oof_preds',axis = 1, inplace = True)

0.846699634196055

In [25]:
#goss
# cols_to_drop = ['aff','peptide','core','LPFR','RPFR','true_index','n_label', 'core_P1', 'core_P2', 'core_P3', 'core_P4','core_P5', 'core_P6', 'core_P7', 'core_P8', 'core_P9']
df['oof_preds'] = oof_preds
idx = df.groupby(['true_index'])['oof_preds'].transform(max) == df['oof_preds']
k = df[idx]
df.drop(columns = 'oof_preds', inplace = True)
from sklearn.metrics import roc_auc_score
roc_auc_score(k['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0), k['oof_preds'])
#df.drop(columns='oof_preds',axis = 1, inplace = True)

0.8489972041409461

In [17]:
#gbdt
df['oof_preds'] = oof_preds
idx = df.groupby(['true_index'])['oof_preds'].transform(max) == df['oof_preds']
k = df[idx]
df.drop(columns = 'oof_preds', inplace = True)
from sklearn.metrics import roc_auc_score
roc_auc_score(k['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0), k['oof_preds'])
#df.drop(columns='oof_preds',axis = 1, inplace = True)

0.8537717807422511

In [26]:
k.drop(columns = ['aff', 'true_index','n_label']).to_csv('stage2_input.csv',index = False)

In [15]:
a = feature_importance_df.groupby('feature').agg({'importance':'mean'})

In [18]:
pd.options.display.max_rows = 100
a.sort_values(by = 'importance')

,importance
feature,
allele_type,3.564324e+03
len_peptide_transform,2.315711e+04
len_LPFR,4.922416e+04
core_P3_std,6.487726e+04
core_P7,6.917599e+04
core_P8,7.031290e+04
core_P5_std,7.104426e+04
core_P5,7.144126e+04
core_P4_std,7.283346e+04


In [31]:
df.head()

,true_index,peptide,hla,aff,allele_type,length,n_label,core,LPFR,RPFR,len_LPFR,len_RPFR,len_peptide_transform
0,1,AAAAAAAAAAA,37,0.324088,1,11,0,AAAAAAAAA,,AA,0.000000,0.666667,0.880797
1,1,AAAAAAAAAAA,37,0.324088,1,11,1,AAAAAAAAA,A,A,0.333333,0.333333,0.880797
2,1,AAAAAAAAAAA,37,0.324088,1,11,2,AAAAAAAAA,AA,,0.666667,0.000000,0.880797
3,2,AAAAAGTTVYGAFAA,46,0.129502,2,15,0,AAAAAGTTV,,YGA,0.000000,1.000000,0.500000
4,2,AAAAAGTTVYGAFAA,46,0.129502,2,15,1,AAAAGTTVY,A,GAF,0.333333,1.000000,0.500000


In [75]:
k.to_csv('stage2_input3.csv', index = False)

In [82]:
k[k['true_index'] == 75918]

,true_index,peptide,hla,aff,allele_type,length,n_label,core,LPFR,RPFR,len_LPFR,len_RPFR,len_peptide_transform,core_P1,core_P2,core_P3,core_P4,core_P5,core_P6,core_P7,core_P8,core_P9,LPFR_P1,LPFR_P2,LPFR_P3,RPFR_P1,RPFR_P2,RPFR_P3,core_a_tfidf,core_c_tfidf,core_d_tfidf,core_e_tfidf,core_f_tfidf,core_g_tfidf,core_h_tfidf,core_i_tfidf,core_k_tfidf,core_l_tfidf,core_m_tfidf,core_n_tfidf,core_p_tfidf,core_q_tfidf,core_r_tfidf,core_s_tfidf,core_t_tfidf,core_v_tfidf,core_w_tfidf,core_y_tfidf,cluster,hla_mean,hla_std,allele_type_mean,allele_type_std,core_P1_mean,core_P1_std,core_P2_mean,core_P2_std,core_P3_mean,core_P3_std,core_P4_mean,core_P4_std,core_P5_mean,core_P5_std,core_P6_mean,core_P6_std,core_P7_mean,core_P7_std,core_P8_mean,core_P8_std,core_P9_mean,core_P9_std,LPFR_P1_mean,LPFR_P1_std,LPFR_P2_mean,LPFR_P2_std,LPFR_P3_mean,LPFR_P3_std,RPFR_P1_mean,RPFR_P1_std,RPFR_P2_mean,RPFR_P2_std,RPFR_P3_mean,RPFR_P3_std,cluster_mean,cluster_std,peptide_nunique,unique_rate,p1,p2,p3,p4,p5,p6,p7,p8,p9,blosum_sum,oof_preds
553288,75918,LQLQPFPQPQLPYPQPQLPYPQPQLPYPQPQPF,73,0.273745,3,33,10,LPYPQPQLP,QPQ,YPQ,1.0,1.0,0.000123,9,12,19,12,13,12,13,9,12,14,13,14,20,13,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.305176,0.0,0.0,0.8125,0.442139,0.0,0.0,0.0,0.0,0.0,0.225952,58,0.314281,0.204903,0.325156,0.230245,0.410244,0.257913,0.313028,0.250265,0.409004,0.264242,0.321826,0.25468,0.37623,0.264482,0.3514,0.26912,0.377549,0.261341,0.413677,0.262606,0.373957,0.272327,0.390232,0.249063,0.323595,0.249211,0.374778,0.250029,0.361677,0.252535,0.390704,0.274232,0.340601,0.247934,0.209152,0.231298,5,0.151515,-9,17,-9,17,1,17,1,-9,17,43,0.000112
553295,75918,LQLQPFPQPQLPYPQPQLPYPQPQLPYPQPQPF,73,0.273745,3,33,17,LPYPQPQLP,QPQ,YPQ,1.0,1.0,0.000123,9,12,19,12,13,12,13,9,12,14,13,14,20,13,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.305176,0.0,0.0,0.8125,0.442139,0.0,0.0,0.0,0.0,0.0,0.225952,58,0.314281,0.204903,0.325156,0.230245,0.410244,0.257913,0.313028,0.250265,0.409004,0.264242,0.321826,0.25468,0.37623,0.264482,0.3514,0.26912,0.377549,0.261341,0.413677,0.262606,0.373957,0.272327,0.390232,0.249063,0.323595,0.249211,0.374778,0.250029,0.361677,0.252535,0.390704,0.274232,0.340601,0.247934,0.209152,0.231298,5,0.151515,-9,17,-9,17,1,17,1,-9,17,43,0.000112
